# Segmenting and Clustering Neighborhoods in Toronto - Part 3
Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown in the coursera website.

I did mention the steps below as to how I proceeded to create the final pandas Dataframe

## PART-1 The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [10]:
import numpy as np # library to handle data in a vectorized manner

import json # library to handle JSON files
import requests
from pandas.io.json import json_normalize

import requests

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import folium # map rendering library

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print('Libraries imported.')

Libraries imported.


In [11]:
tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)

# I did not use Beatiful Soup for this as I felt this would be easier and faster
required_cols = ['Postal Code', 'Borough','Neighborhood']

# Note I converted the boolean output of array_equal function to String as it 
# wasn't working as expected when used in the if condition as expected
for table in tables:
    if(str(np.array_equal(np.array(table.columns),np.array(required_cols)))=="True"):
        pstl_data_df = pd.DataFrame(table)    
    break
print("Shape of Dataframe is - ",pstl_data_df.shape)
pstl_data_df.head()

Shape of Dataframe is -  (180, 3)


,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Step 2: Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [12]:
#Filtering out Boroughs which are Not assigned
pstl_data_df = pstl_data_df[pstl_data_df.Borough!="Not assigned"]
print("Shape of Dataframe is - ",pstl_data_df.shape)
pstl_data_df.head()

Shape of Dataframe is -  (103, 3)


,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Step 3: If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [13]:
pstl_data_df['new_nghbr'] = np.where(pstl_data_df['Neighborhood']=='Not assigned',pstl_data_df['Borough'],pstl_data_df['Neighborhood'])
pstl_data_df.head(10)

,Postal Code,Borough,Neighborhood,new_nghbr
2,M3A,North York,Parkwoods,Parkwoods
3,M4A,North York,Victoria Village,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront","Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights","Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government","Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village","Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge","Malvern, Rouge"
11,M3B,North York,Don Mills,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens","Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson","Garden District, Ryerson"


### Step 4: More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
Used the "new_nghbr" column to get a grouped column, as in wherever the same Postal Code and the same Borough are present we have to club the Neighbourhood column using the functions groupby and apply

In [14]:
can_postal_cd_df = pd.DataFrame(pstl_data_df.groupby(['Postal Code','Borough'])['new_nghbr'].apply(','.join).reset_index())

# Renaming Columns to the same one in the image given:
can_postal_cd_df = can_postal_cd_df.rename(columns = {"Postal Code": "PostalCode","new_nghbr":"Neighborhood"})

In [15]:
can_postal_cd_df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [16]:
print("\n The Final Shape of the dataframe is  - ",can_postal_cd_df.shape)


 The Final Shape of the dataframe is  -  (103, 3)


## PART-2 Add the Lat Long data to the existing dataframe

In [ ]:
# Installing GeoCoder
!conda install -c conda-forge geocoder --yes
print("Installation Done!")
import geocoder # import geocoder
print("Geo Coder imported!")

Solving environment: / 

In [17]:
#Creating a function to get the Lat Long data from the Postal Code
def get_geocoder(postal_code_from_df):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code_from_df.strip()))
        lat_lng_coords = g.latlng
        latitude = lat_lng_coords[0]
        longitude = lat_lng_coords[1]
    return latitude,longitude

In [18]:
#Adding the Latitude and Longitude columns to the Pandas DataFrame
can_postal_cd_df['Latitude'], can_postal_cd_df['Longitude'] = zip(*can_postal_cd_df['PostalCode'].apply(get_geocoder))
can_postal_cd_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.81153,-79.19552
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.78564,-79.15871
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.76575,-79.17520
3,M1G,Scarborough,Woburn,43.76820,-79.21761
4,M1H,Scarborough,Cedarbrae,43.76969,-79.23944


In [19]:
print("Shape of the dataframe is - ",can_postal_cd_df.shape)

Shape of the dataframe is -  (103, 5)


### Map of Toronto
Using geolocator for Mapping Toronto

In [22]:
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


Solving environment: done

# All requested packages already installed.



In [23]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_ontario")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto, Ontario are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto, Ontario are 43.6534817, -79.3839347.


In [24]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, long, post, borough, neigh in zip(can_postal_cd_df['Latitude'], can_postal_cd_df['Longitude'], can_postal_cd_df['PostalCode'], can_postal_cd_df['Borough'], can_postal_cd_df['Neighborhood']):
    label = "{} ({}): {}".format(borough, post, neigh)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

## PART-3 Explore and Cluster Neighbourhoods in Toronto
#### Step - 1: Filtering out Toronto Boroughs in the complete Data

In [25]:
# Printing all Borough values in the Pandas Dataframe
can_postal_cd_df.Borough.unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       'Mississauga', 'Etobicoke'], dtype=object)

In [26]:
toronto_boroughs = ['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']
toronto_all_df = can_postal_cd_df[can_postal_cd_df['Borough'].isin(toronto_boroughs)].reset_index(drop=True)
print("Shape of All Toronto Data dataframe is - ",toronto_all_df.shape)
toronto_all_df.head(40)

Shape of All Toronto Data dataframe is -  (39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.67703,-79.29542
1,M4K,East Toronto,"The Danforth West, Riverdale",43.68375,-79.35528
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.66797,-79.31468
3,M4M,East Toronto,Studio District,43.66091,-79.33503
4,M4N,Central Toronto,Lawrence Park,43.72898,-79.39173
5,M4P,Central Toronto,Davisville North,43.71276,-79.38851
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.71452,-79.40696
7,M4S,Central Toronto,Davisville,43.70340,-79.38596
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.69066,-79.38356
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.68569,-79.40232


In [27]:
#Mapping only Toronto Data which was filtered earlier, we can see the variance in both maps which had the locations marked
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, long, post, borough, neigh in zip(toronto_all_df['Latitude'], toronto_all_df['Longitude'], toronto_all_df['PostalCode'], toronto_all_df['Borough'], toronto_all_df['Neighborhood']):
    label = "{} ({}): {}".format(borough, post, neigh)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
map_toronto

#### Step - 2:Using Foursquare API going forward:

In [29]:
# The code was removed by Watson Studio for sharing.

#### All recommended places in each Borough in Toronto

In [31]:
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
radius = 500
LIMIT = 1000

recommends = []
for lat, long, post, borough, neighborhood in zip(toronto_all_df['Latitude'], toronto_all_df['Longitude'], toronto_all_df['PostalCode'], toronto_all_df['Borough'], toronto_all_df['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    postal_data = requests.get(url).json()["response"]['groups'][0]['items']
    for recommend_post in postal_data:
        recommends.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            recommend_post['venue']['name'], 
            recommend_post['venue']['location']['lat'], 
            recommend_post['venue']['location']['lng'],  
            recommend_post['venue']['categories'][0]['name']))

In [32]:
toronto_recommends = pd.DataFrame(recommends)
toronto_recommends.columns = ['PostalCode', 'Borough', 'Neighborhoods', 'Borough_Lat', 'Borough_Long', 'Venue_Name', 'Venue_Lat', 'Venue_Long', 'Venue_Category']
print("Shape of all the recommendations of all the Postal Codes in Toronto - ", toronto_recommends.shape)
toronto_recommends.head()

Shape of all the recommendations of all the Postal Codes in Toronto -  (1754, 9)


,PostalCode,Borough,Neighborhoods,Borough_Lat,Borough_Long,Venue_Name,Venue_Lat,Venue_Long,Venue_Category
0,M4E,East Toronto,The Beaches,43.67703,-79.29542,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,East Toronto,The Beaches,43.67703,-79.29542,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,East Toronto,The Beaches,43.67703,-79.29542,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,East Toronto,The Beaches,43.67703,-79.29542,Seaspray Restaurant,43.678888,-79.298167,Asian Restaurant
4,M4E,East Toronto,The Beaches,43.67703,-79.29542,Upper Beaches,43.680563,-79.292869,Neighborhood


In [33]:
#Removing any duplicates if any
toronto_recommends.drop_duplicates(keep=False, inplace=True)
toronto_recommends.shape

(1754, 9)

In [34]:
toronto_recommends.groupby(['PostalCode', 'Borough', 'Neighborhoods'])['Venue_Name'].count()

PostalCode  Borough           Neighborhoods                                                                                             
M4E         East Toronto      The Beaches                                                                                                     5
M4K         East Toronto      The Danforth West, Riverdale                                                                                    6
M4L         East Toronto      India Bazaar, The Beaches West                                                                                 20
M4M         East Toronto      Studio District                                                                                                51
M4N         Central Toronto   Lawrence Park                                                                                                   2
M4P         Central Toronto   Davisville North                                                                                                8

In [35]:
print("Number of Unique Venue Categories are: ",len(toronto_recommends['Venue_Category'].unique()))
toronto_recommends['Venue_Category'].unique()

Number of Unique Venue Categories are:  225


array(['Trail', 'Health Food Store', 'Pub', 'Asian Restaurant',
       'Neighborhood', 'Ice Cream Shop', 'Discount Store', 'Park',
       'Intersection', 'Bus Line', 'Grocery Store', 'Gym',
       'Fast Food Restaurant', 'Fish & Chips Shop', 'Sushi Restaurant',
       'Italian Restaurant', 'Burrito Place', 'Pet Store', 'Steakhouse',
       'Liquor Store', 'Restaurant', 'Movie Theater', 'Sandwich Place',
       'Board Shop', 'Pizza Place', 'Fruit & Vegetable Store',
       'Latin American Restaurant', 'Bakery', 'Boutique', 'Coffee Shop',
       'Vietnamese Restaurant', 'Brewery', 'Café', 'American Restaurant',
       'Antique Shop', 'Bar', 'Gay Bar', 'Diner', 'Flea Market',
       'French Restaurant', 'Arts & Crafts Store',
       'Comfort Food Restaurant', 'Seafood Restaurant', 'Thai Restaurant',
       'Gastropub', 'Convenience Store', 'Clothing Store',
       'Deli / Bodega', 'Light Rail Station', 'Wine Shop',
       'Electronics Store', 'Breakfast Spot', 'Food & Drink Shop',
       

### Analyze venues in each area

In [36]:
# getting all the values as columns for all areas
toronto_recommends = toronto_recommends.drop(['Borough_Lat','Borough_Long','Venue_Lat','Venue_Long'],axis=1)
toronto_recommends_df = pd.get_dummies(toronto_recommends, columns=['Venue_Category'],prefix = "", prefix_sep = "")
toronto_recommends_df.head(40)

,PostalCode,Borough,Neighborhoods,Venue_Name,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recreation Center,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Strip Club,Summer Camp,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,Glen Manor Ravine,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,M4E,East Toronto,The Beaches,The Big Carrot Natural Food Market,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [37]:
toronto_venues_freq = toronto_recommends_df.groupby(['PostalCode','Borough','Neighborhoods']).mean().reset_index()
toronto_venues_freq.head(40)

,PostalCode,Borough,Neighborhoods,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recreation Center,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Strip Club,Summer Camp,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000

### Top 10 Venues in the each of the Areas

In [39]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# Create a new dataframe
all_toronto_venues = pd.DataFrame(columns=columns)
all_toronto_venues['PostalCode'] = toronto_venues_freq['PostalCode']
all_toronto_venues['Borough'] = toronto_venues_freq['Borough']
all_toronto_venues['Neighborhoods'] = toronto_venues_freq['Neighborhoods']
for ind in np.arange(toronto_venues_freq.shape[0]):
    row_categories = toronto_venues_freq.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    all_toronto_venues.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

all_toronto_venues.sort_values(freqColumns, inplace=True)
all_toronto_venues

,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
34,M6R,West Toronto,"Parkdale, Roncesvalles",Bakery,Eastern European Restaurant,Coffee Shop,Pizza Place,Restaurant,Thai Restaurant,Café,Bookstore,Sushi Restaurant,Gift Shop
32,M6K,West Toronto,"Brockton, Parkdale Village, Exhibition Place",Café,Bar,Coffee Shop,Restaurant,Gift Shop,Sandwich Place,Nightclub,Supermarket,Italian Restaurant,Furniture / Home Store
24,M5S,Downtown Toronto,"University of Toronto, Harbord",Café,Coffee Shop,Bookstore,Bakery,Restaurant,Gym,Japanese Restaurant,Bar,Yoga Studio,Pharmacy
25,M5T,Downtown Toronto,"Kensington Market, Chinatown, Grange Park",Café,Coffee Shop,Pizza Place,Vegetarian / Vegan Restaurant,Mexican Restaurant,Park,Burger Joint,Dumpling Restaurant,Grocery Store,Ice Cream Shop
29,M6G,Downtown Toronto,Christie,Café,Grocery Store,Coffee Shop,Italian Restaurant,Playground,Baby Store,Candy Store,Electronics Store,Elementary School,Ethiopian Restaurant
31,M6J,West Toronto,"Little Portugal, Trinity",Cocktail Bar,Coffee Shop,Bar,Asian Restaurant,Restaurant,Wine Bar,Vietnamese Restaurant,Diner,Record Shop,Pizza Place
35,M6S,West Toronto,"Runnymede, Swansea",Coffee Shop,Café,Bakery,Pizza Place,Bank,Bookstore,Pet Store,Pharmacy,Pilates Studio,Italian Restaurant
28,M5X,Downtown Toronto,"First Canadian Place, Underground city",Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Gym,Deli / Bodega,Japanese Restaurant,Seafood Restaurant,Asian Restaurant
14,M5B,Downtown Toronto,"Garden District, Ryerson",Coffee Shop,Clothing Store,Hotel,Italian Restaurant,Café,Japanese Restaurant,Cosmetics Shop,Ramen Restaurant,Bookstore,Lingerie Store
17,M5G,Downtown Toronto,Central Bay Street,Coffee Shop,Clothing Store,Plaza,Electronics Store,Sandwich Place,Bubble Tea Shop,Pizza Place,Middle Eastern Restaurant,Cosmetics Shop,Hotel


### Clustering areas
#### Use KMeans algorigthm, try to cluster the toronto central areas into 5 clusters

In [41]:
#from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

toronto_venues_freq_clustering = toronto_venues_freq.drop(['PostalCode', 'Borough', 'Neighborhoods'], 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_venues_freq_clustering)

#No data for a particular postal code so removed it
toronto_clustered_df = toronto_all_df.drop(toronto_all_df.index[toronto_all_df.PostalCode=='M5N'])
toronto_clustered_df['Cluster'] = kmeans.labels_

toronto_clustered_df = toronto_clustered_df.join(all_toronto_venues.drop(['Borough', 'Neighborhoods'], 1).set_index('PostalCode'), on='PostalCode')
toronto_clustered_df.sort_values(['Cluster'] + freqColumns, inplace=True)
toronto_clustered_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,M4P,Central Toronto,Davisville North,43.71276,-79.38851,0,Convenience Store,Hotel,Park,Department Store,Food & Drink Shop,Breakfast Spot,American Restaurant,Gym / Fitness Center,Gym,Dumpling Restaurant
34,M6P,West Toronto,"High Park, The Junction South",43.65994,-79.46302,0,Convenience Store,Park,Residential Building (Apartment / Condo),Donut Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.71452,-79.40696,0,Gym Pool,Playground,Park,Garden,Yoga Studio,Dog Run,Farmers Market,Farm,Falafel Restaurant,Event Space
1,M4K,East Toronto,"The Danforth West, Riverdale",43.68375,-79.35528,0,Ice Cream Shop,Intersection,Bus Line,Grocery Store,Park,Discount Store,Elementary School,Ethiopian Restaurant,Event Space,Falafel Restaurant
10,M4W,Downtown Toronto,Rosedale,43.68190,-79.37850,0,Tennis Court,Gym / Fitness Center,Playground,Park,Dog Run,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
23,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.69479,-79.41440,1,Park,Yoga Studio,Dumpling Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.69066,-79.38356,2,Gym,Summer Camp,Yoga Studio,Donut Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
4,M4N,Central Toronto,Lawrence Park,43.72898,-79.39173,3,Park,Electronics Store,Yoga Studio,Dumpling Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
35,M6R,West Toronto,"Parkdale, Roncesvalles",43.64785,-79.45020,4,Bakery,Eastern European Restaurant,Coffee Shop,Pizza Place,Restaurant,Thai Restaurant,Café,Bookstore,Sushi Restaurant,Gift Shop
33,M6K,West Toronto,"Brockton, Parkdale Village, Exhibition Place",43.63941,-79.42676,4,Café,Bar,Coffee Shop,Restaurant,Gift Shop,Sandwich Place,Nightclub,Supermarket,Italian Restaurant,Furniture / Home Store


#### Mapping clusters

In [43]:
#import matplotlib.cm as cm
#import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_clustered_df['Latitude'], toronto_clustered_df['Longitude'], toronto_clustered_df['PostalCode'], toronto_clustered_df['Borough'], toronto_clustered_df['Neighborhood'], toronto_clustered_df['Cluster']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters